# Flask: WebApp using sqlite3

Prerequistes:
*    pip install flask
*    pip install flask_sqlalchemy
*    pip install webdriver_manager

In [1]:
cd '/home/chris/workspace/python-course/Level 4/02 Flask'
ls -l
rm -rf workspace3
ls -l

total 88
-rw-rw-r-- 1 chris chris  6377 Dec  7 12:14 03.ipynb
-rw-rw-r-- 1 chris chris  8359 Dec  4 22:18 1.basic_webapp.ipynb
-rw-rw-r-- 1 chris chris 28450 Dec  6 23:44 2.database_webapp2.ipynb
-rw-rw-r-- 1 chris chris 23904 Dec  7 22:36 3.database_webapp.ipynb
-rw-rw-r-- 1 chris chris  1410 Dec  7 00:10 database_app.py
drwxrwxr-x 2 chris chris  4096 Dec  7 14:43 __pycache__
drwxrwxr-x 5 chris chris  4096 Dec  7 22:37 workspace3
-rw-rw-r-- 1 chris chris    30 Dec  7 14:41 z.html
total 84
-rw-rw-r-- 1 chris chris  6377 Dec  7 12:14 03.ipynb
-rw-rw-r-- 1 chris chris  8359 Dec  4 22:18 1.basic_webapp.ipynb
-rw-rw-r-- 1 chris chris 28450 Dec  6 23:44 2.database_webapp2.ipynb
-rw-rw-r-- 1 chris chris 23904 Dec  7 22:36 3.database_webapp.ipynb
-rw-rw-r-- 1 chris chris  1410 Dec  7 00:10 database_app.py
drwxrwxr-x 2 chris chris  4096 Dec  7 14:43 __pycache__
-rw-rw-r-- 1 chris chris    30 Dec  7 14:41 z.html


Create a workspace for our project

In [2]:
stty -echo
mkdir workspace3
cd workspace3

Create a basic webapp

In [3]:
# create new app
cat << EOF > database_app.py
import os
from flask import Flask, render_template, request, url_for, redirect
from flask_sqlalchemy import SQLAlchemy

from sqlalchemy.sql import func


basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] =\
        'sqlite:///' + os.path.join(basedir, 'database.db')
#app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Match(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    homeTeam = db.Column(db.String(100), nullable=False)
    homeScore = db.Column(db.Integer)
    awayTeam = db.Column(db.String(100), nullable=False)
    awayScore = db.Column(db.Integer)

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
        
    def __str__(self):
        return f"{self.homeTeam} {self.homeScore}-{self.awayScore} {self.awayTeam}"

@app.route('/')
def index():
    matches = Match.query.all()
    return render_template('index.html', matches=matches)

EOF

Run the Flask shell program to create the in-memory database

In [4]:
export FLASK_APP=database_app
flask shell << EOF
from database_app import db, Match
db.drop_all()
db.create_all()
EOF

Python 3.10.15 (main, Oct 18 2024, 15:04:49) [GCC 13.2.0] on linux
App: database_app
Instance: /home/chris/workspace/python-course/src/53 Flask/workspace3/instance
>>> >>> >>> >>> 
now exiting InteractiveConsole...


The project structure is now:

In [5]:
tree -I __pycache__ .

.
├── database_app.py
└── database.db

1 directory, 2 files


Now run the Flask shell program to populate the database with some example entries

In [6]:
export FLASK_APP=database_app
flask shell << EOF

def playMatch(home, score1, away, score2):
    m = Match()
    m.play(home, score1, away, score2)
    return m

m1 = playMatch("Red", 2, "Blue", 1)
m2 = playMatch("Green", 5, "White", 0)
m3 = playMatch("Red", 3, "Green", 1)

db.session.add(m1)
db.session.add(m2)
db.session.add(m3)
db.session.commit()
Match.query.all()
EOF

Python 3.10.15 (main, Oct 18 2024, 15:04:49) [GCC 13.2.0] on linux
App: database_app
Instance: /home/chris/workspace/python-course/src/53 Flask/workspace3/instance
>>> >>> ... ... ... ... >>> >>> >>> >>> >>> >>> >>> >>> >>> [<Match 1>, <Match 2>, <Match 3>]
>>> 
now exiting InteractiveConsole...


Create a `base` template using the `jinja2` template engine.   Our actual templates will inherit from this template.

In [7]:
mkdir -p templates
cat << EOF > templates/base.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}"    
</head>
<body>
    <h1>{% block title %} {% endblock %}</h1>
    <div>
        {% block contents %} {% endblock %}
    </div>
</body>
</html>
EOF

Now create an real template.  The base template defines the basic structure, but we need to define the `block content` section here.

In [8]:
mkdir -p templates
cat << EOF > templates/index.html
{% extends 'base.html' %}
{% block title %}
RESULTS
{% endblock %}
{% block contents %}
    <table>
    {% for match in matches %}
        <tr><td>{{ match }}</td></tr>
    {% endfor %}
    </table>
    <a href="http://localhost:8000/create">create</a>
{% endblock %}
EOF

Create a stylesheet - this must be stored in the `static` subdirectory

In [9]:
mkdir -p static/styles
cat << EOF > static/styles/mystyles.css
.title {
    margin: 5px;
}

body {background-color: powderblue;}
table, th, td {
  border: 1px solid;
}
EOF

Start the Flask development server:

In [10]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

[1] 7261


Now display the index view in firefox

In [11]:
firefox http://localhost:8000

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit


Now create a route to display a `single match`:

In [12]:
cat << EOF >> database_app.py
@app.route('/<int:match_id>/')
def singleMatch(match_id):
    match = Match.query.get_or_404(match_id)
    return render_template('singleMatch.html', match=match, id=match_id)
EOF

127.0.0.1 - - [09/Dec/2024 14:29:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2024 14:29:29] "GET /static/styles/mystyles.css HTTP/1.1" 200 -


Add the corresponding template:

In [13]:
cat << EOF > templates/singleMatch.html
{% extends 'base.html' %}
{% block title %}
    MATCH {{ id }}
{% endblock %}
{% block contents %}
    <div class="content">
    <div>{{ match }}</div>
{% endblock %}
EOF

Project structure is now:

In [14]:
tree -I __pycache__ .

.
├── database_app.py
├── database.db
├── static
│   └── styles
│       └── mystyles.css
└── templates
    ├── base.html
    ├── index.html
    └── singleMatch.html

4 directories, 6 files


Restart the server:

In [15]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

8000/tcp:             7261
[2] 7578
[1]   Killed                  flask --app database_app run --host localhost --port 8000


Display the 2nd entry in the browser:

In [16]:
firefox http://localhost:8000/2

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit


Add a route to allow us to `create` new entries in the database:

In [17]:
cat << EOF >> database_app.py

@app.route('/create/', methods=('GET', 'POST'))
@app.route('/create/', methods=('GET', 'POST'))
def create():
    if request.method == 'POST':
        homeTeam = request.form['homeTeam']
        awayTeam = request.form['awayTeam']
        homeScore = request.form['homeScore']
        awayScore = request.form['awayScore']

        match = Match()
        match.play(homeTeam, homeScore, awayTeam, awayScore)
        db.session.add(match)
        db.session.commit()

        return redirect(url_for('index'))

    return render_template('create.html')
EOF

127.0.0.1 - - [09/Dec/2024 14:29:50] "GET /2 HTTP/1.1" 308 -
127.0.0.1 - - [09/Dec/2024 14:29:50] "GET /2/ HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2024 14:29:51] "GET /static/styles/mystyles.css HTTP/1.1" 304 -


Add the template for the route:

In [18]:
cat << EOF > templates/create.html
{% extends 'base.html' %}

{% block title %}
    <h1>Add Match {{ id }}</h1>
{% endblock %}

{% block contents %}
    <form method="post">
        <p>
            <label for="homeTeam">Home Team</label>
            <input id="h" type="text" name="homeTeam" placeholder="home team"/>
        </p>
        <p>
            <label for="homeScore">Home Score</label>
            <input type="text" name="homeScore" placeholder="home score"/>
        </p>
        <p>
            <label for="awayTeam">Away Team</label>
            <input type="text" name="awayTeam" placeholder="away team"/>
        </p>
        <p>
            <label for="awayScore">Away Score</label>
            <input type="text" name="awayScore" placeholder="away score"/>
        </p>
        <p>
            <button id="submit" type="submit">Submit</button>
        </p>
    </form>
{% endblock %}
EOF

In [19]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

8000/tcp:             7578
[3] 7847
[2]   Killed                  flask --app database_app run --host localhost --port 8000


In [20]:
firefox http://localhost:8000/create

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit


Now create a `Selenium` script to automate filling in the `create` form: 

In [21]:
cat << EOF > go.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

driver = webdriver.Chrome()
driver = webdriver.Firefox()

driver.get("http://localhost:8000/create")

time.sleep(5)
title = driver.title

driver.implicitly_wait(0.5)

def fillInForm(h, hs, a, awayScore):
    text_box = driver.find_element(by=By.NAME, value="homeTeam")
    text_box.send_keys(h)
    text_box = driver.find_element(by=By.NAME, value="homeScore")
    text_box.send_keys(hs)
    text_box = driver.find_element(by=By.NAME, value="awayTeam")
    text_box.send_keys(a)
    text_box = driver.find_element(by=By.NAME, value="awayScore")
    text_box.send_keys(awayScore)
    driver.implicitly_wait(5)
    current_url = driver.current_url
    submit_button = driver.find_element(by=By.ID, value="submit")
    submit_button.click()    
    WebDriverWait(driver, 15).until(EC.url_changes(current_url))
    time.sleep(5)

fillInForm("White", 5, "Black", 3)
link = driver.find_element(By.LINK_TEXT, 'create')
link.click()
fillInForm("Purple", 4, "Red", 2)
link = driver.find_element(By.LINK_TEXT, 'create')
link.click()
fillInForm("Green", 5, "Blue", 0)
link = driver.find_element(By.LINK_TEXT, 'create')
link.click()

time.sleep(50)

driver.quit()
EOF

127.0.0.1 - - [09/Dec/2024 14:30:04] "GET /create HTTP/1.1" 308 -
127.0.0.1 - - [09/Dec/2024 14:30:04] "GET /create/ HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2024 14:30:04] "GET /static/styles/mystyles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 14:30:09] "POST /create/ HTTP/1.1" 302 -
127.0.0.1 - - [09/Dec/2024 14:30:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2024 14:30:09] "GET /static/styles/mystyles.css HTTP/1.1" 304 -


In [22]:
export TMPDIR=/tmp
python go.py

Traceback (most recent call last):
  File "/home/chris/workspace/python-course/src/53 Flask/workspace3/go.py", line 9, in <module>
    driver = webdriver.Firefox()
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/selenium/webdriver/firefox/webdriver.py", line 71, in __init__
    super().__init__(command_executor=executor, options=options)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 241, in __init__
    self.start_session(capabilities)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 329, in start_session
    response = self.execute(Command.NEW_SESSION, caps)["value"]
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 384, in execute
    self.error_handler.check_response(response)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/s

: 1

In [248]:
cd ..
fuser -k 7000/tcp
rm -r workspace4

rm: cannot remove 'workspace4': No such file or directory


: 1

Create the template.  This needs to be stored in the `template` subdirectory.  
Note how we link to the stylesheet:  
```    <link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}">```

The above expands to:  
```    <link rel="stylesheet" type= "text/css" href= "/static/styles/mystyles.css">```